In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy as sps
import csv
#import pylab as plt
#import qutip as qt

In [2]:
def Hamiltonian(kx, ky, theta):
    
    theta  = theta/180.0*np.pi 
    b1m    = 8*np.pi*np.sin(theta/2)/3/d*np.array([0.5, -np.sqrt(3)/2])
    b2m    = 8*np.pi*np.sin(theta/2)/3/d*np.array([0.5, np.sqrt(3)/2])
    qb     = 8*np.pi*np.sin(theta/2)/3/np.sqrt(3)/d*np.array([0, -1])
    K1     = 8*np.pi*np.sin(theta/2)/3/np.sqrt(3)/d*np.array([-np.sqrt(3)/2,-0.5])
    K2     = 8*np.pi*np.sin(theta/2)/3/np.sqrt(3)/d*np.array([-np.sqrt(3)/2,0.5])
    H = np.array(np.zeros((4*siteN, 4*siteN)), dtype=complex)
    
    for i in np.arange(siteN):
        #diagonal term
        ix = L[i, 0]
        iy = L[i, 1]
        ax = kx - valley*K1[0] + ix*b1m[0] + iy*b2m[0]
        ay = ky - valley*K1[1] + ix*b1m[1] + iy*b2m[1]

        qx = np.cos(theta/2) * ax + np.sin(theta/2) * ay
        qy =-np.sin(theta/2) * ax + np.cos(theta/2) * ay
         
        H[2*i, 2*i+1] = hv * (valley*qx - I*qy)
        H[2*i+1, 2*i] = hv * (valley*qx + I*qy)

        #off-diagonal term
        j = i + siteN
        H[2*j, 2*i]     = TqbD[0, 0]
        H[2*j, 2*i+1]   = TqbD[0, 1]
        H[2*j+1, 2*i]   = TqbD[1, 0]
        H[2*j+1, 2*i+1] = TqbD[1, 1]
        if (iy != valley*N and ix != valley*N):
            j = invL[ix+1+N, iy+valley*1+N] + siteN
            H[2*j, 2*i]     = TqtrD[0, 0]
            H[2*j, 2*i+1]   = TqtrD[0, 1]
            H[2*j+1, 2*i]   = TqtrD[1, 0]
            H[2*j+1, 2*i+1] = TqtrD[1, 1]
        if (ix != valley*N):
            j = invL[ix+valley*1+N, iy+N] + siteN
            H[2*j, 2*i]     = TqtlD[0, 0]
            H[2*j, 2*i+1]   = TqtlD[0, 1]
            H[2*j+1, 2*i]   = TqtlD[1, 0]
            H[2*j+1, 2*i+1] = TqtlD[1, 1]
        

    for i in np.arange(siteN, 2*siteN):
        #diagnoal term
        j = i - siteN
        ix = L[j, 0]
        iy = L[j, 1]
        ax = kx  - valley*K2[0] + ix*b1m[0] + iy*b2m[0] 
        ay = ky  - valley*K2[1] + ix*b1m[1] + iy*b2m[1]

        qx = np.cos(theta/2) * ax - np.sin(theta/2) * ay
        qy = np.sin(theta/2) * ax + np.cos(theta/2) * ay

        H[2*i, 2*i+1] = hv * (valley*qx - I*qy)
        H[2*i+1, 2*i] = hv * (valley*qx + I*qy)

        #off-diagonal term
        H[2*j, 2*i]     = Tqb[0, 0]
        H[2*j, 2*i+1]   = Tqb[0, 1]
        H[2*j+1, 2*i]   = Tqb[1, 0]
        H[2*j+1, 2*i+1] = Tqb[1, 1]
        if (iy != (-valley*N) and ix != (-valley*N)):
            j = invL[ix-1+N, iy-valley*1+N]
            H[2*j, 2*i]     = Tqtr[0, 0]
            H[2*j, 2*i+1]   = Tqtr[0, 1]
            H[2*j+1, 2*i]   = Tqtr[1, 0]
            H[2*j+1, 2*i+1] = Tqtr[1, 1]
        if (ix != -valley*N):
            j = invL[ix-valley*1+N, iy+N]
            H[2*j, 2*i]     = Tqtl[0, 0]
            H[2*j, 2*i+1]   = Tqtl[0, 1]
            H[2*j+1, 2*i]   = Tqtl[1, 0]
            H[2*j+1, 2*i+1] = Tqtl[1, 1]


    eigensystem = sps.sparse.linalg.eigsh(H, k = levels, which = "SM")
    
    return eigensystem

In [ ]:
kD = -qb[1]

AtoB = np.arange(-1/2, 1/2, 1/KDens)
BtoC = np.arange(-1, 0, 1/KDens)
CtoD = np.arange(0, sqrt(3), 1/KDens)
DtoA = np.arange(0, 1, 1/KDens)

AllK  = len(AtoB) + len(BtoC) + len(CtoD) + len(DtoA)
E  = np.zeros((AllK,4*siteN), float)


for i in range(0, len(AtoB)):
    k = AtoB[i]
    E[i] = np.real(Hamiltonian(sqrt(3)/2*kD, k*kD))
for i in range(len(AtoB), len(AtoB)+len(BtoC)):
    k = BtoC[i-len(AtoB)]
    E[i] = np.real(Hamiltonian(0, k*kD))
for i in range(len(AtoB)+len(BtoC), len(AtoB)+len(BtoC)+len(CtoD)):
    k = CtoD[i-len(AtoB)-len(BtoC)]
    E[i] = np.real(Hamiltonian(1.0/2*k*kD, -k*sqrt(3)/2*kD))
for i in range(len(AtoB)+len(BtoC)+len(CtoD), AllK):
    k = DtoA[i-len(AtoB)-len(BtoC)-len(CtoD)]
    E[i] = np.real(Hamiltonian(-sqrt(3)/2*k*kD, -1/2*k*kD))


%matplotlib qt

for j in range(0,4*siteN):
    plt.plot(np.arange(AllK), E[:,j], linestyle="-", linewidth=2)
plt.title("Moir$\\'{e}$ bands of twisted bilayer graphene", fontsize=20)
plt.xlim(0, AllK)
plt.ylim(-2000,2000)
plt.yticks(fontsize=13)
plt.ylabel('E(meV)', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
-3.612099949722249459e-01 3.612099949722249459e-01

In [41]:
vals = np.array([-0.1, 0.1])
omega = 0.2
e_win = 0.005
result = np.where(vals>0, 0, vals)
occup = np.where(result<0, 1 ,result)

In [42]:
occup

array([1., 0.])

In [69]:
omega_minus = max(omega-e_win, 1.e-6)
omega_plus = omega + e_win
inds_ef = np.intersect1d(np.where(occup>0)[0], np.where(occup<1)[0])
if len(inds_ef):
    ind_vbm = max(inds_ef)
    ind_cbm = min(inds_ef)
else:
    ind_vbm = max(np.where(occup==1)[0])
    ind_cbm = min(np.where(occup==0)[0])
vbm = vals[ind_vbm]
cbm = vals[ind_cbm]
    
e_bott = cbm - omega - e_win
e_top = vbm + omega + e_win
inds_shot = np.intersect1d(np.where(vals>=e_bott), np.where(vals<=e_top))

inds_vb = np.arange(ind_vbm, inds_shot[0]-1, -1)
inds_cb = np.arange(ind_cbm, inds_shot[-1]+1)


In [71]:
occup

array([1., 0.])

In [44]:
inds_ef

array([], dtype=int64)

In [70]:
ind_vbm

0

In [46]:
ind_cbm

1

In [47]:
e_bott

-0.10500000000000001

In [48]:
e_top

0.10500000000000001

In [49]:
np.intersect1d(np.where(vals>=e_bott), np.where(vals<=e_top))

array([0, 1])

In [50]:
np.where(vals>=e_bott)

(array([0, 1]),)

In [51]:
np.where(vals<=e_top)

(array([0, 1]),)

In [66]:
inds_shot = np.intersect1d(np.where(vals>=e_bott), np.where(vals<=e_top))

In [67]:
inds_shot

array([0, 1])

In [68]:
def add_pair(ind_vb):
    e0 = vals[ind_vb]
    des = vals - e0
    inds_chosen = np.intersect1d(np.where(des>=omega_minus)[0], np.where(des<=omega_plus)[0])
    inds_chosen = np.intersect1d(inds_chosen, inds_cb)
    pairs_chosen = [[ind_vb, indi] for indi in inds_chosen]
    return pairs_chosen
pairs = [add_pair(ind_vb) for ind_vb in inds_vb]
pairs = [i for i in pairs if len(i)]

In [55]:
pairs

[[[0, 1]]]

In [56]:
inds_vb

array([0])

In [32]:
e0

NameError: name 'e0' is not defined

In [57]:
ind_vb = 0
e0 = vals[ind_vb]
des = vals - e0
inds_chosen = np.intersect1d(np.where(des>=omega_minus)[0], np.where(des<=omega_plus)[0])
inds_chosen = np.intersect1d(inds_chosen, inds_cb)
pairs_chosen = [[ind_vb, indi] for indi in inds_chosen]

In [58]:
e0

-0.1

In [59]:
des

array([0. , 0.2])

In [60]:
inds_chosen

array([1])

In [61]:
np.intersect1d(np.where(des>=omega_minus)[0], np.where(des<=omega_plus)[0])

array([1])

In [62]:
omega_minus

0.195

In [63]:
omega_plus

0.20500000000000002

In [65]:
pairs_chosen

[[0, 1]]

In [76]:
def pick_up_transition_pairs(vals, omega, e_win, occup):
    omega_minus = max(omega-e_win, 1.e-6)
    omega_plus = omega + e_win
    inds_ef = np.intersect1d(np.where(occup>0)[0], np.where(occup<1)[0])
    if len(inds_ef):
        ind_vbm = max(inds_ef)
        ind_cbm = min(inds_ef)
    else:
        ind_vbm = max(np.where(occup==1)[0])
        ind_cbm = min(np.where(occup==0)[0])
    vbm = vals[ind_vbm]
    cbm = vals[ind_cbm]
    
    e_bott = cbm - omega - e_win
    e_top = vbm + omega + e_win
    inds_shot = np.intersect1d(np.where(vals>=e_bott), np.where(vals<=e_top))

    if len(inds_shot):
        inds_vb = np.arange(ind_vbm, inds_shot[0]-1, -1)
        inds_cb = np.arange(ind_cbm, inds_shot[-1]+1)
    else:
        inds_vb = []
        inds_cb = []

    def add_pair(ind_vb):
        e0 = vals[ind_vb]
        des = vals - e0
        inds_chosen = np.intersect1d(np.where(des>=omega_minus)[0], np.where(des<=omega_plus)[0])
        inds_chosen = np.intersect1d(inds_chosen, inds_cb)
        pairs_chosen = [[ind_vb, indi] for indi in inds_chosen]
        return pairs_chosen
    pairs = [add_pair(ind_vb) for ind_vb in inds_vb]
#    pairs = [add_pair(ind_vbm)]
    pairs = [i for i in pairs if len(i)]
    if len(pairs):
        return np.concatenate(pairs)
    else:
        return []
    
def occup_0K(vals, spin=1):
    """
    for graphene system, one pz orbital contribute one electron
    """
    result = np.where(vals>0, 0, vals)
    result2 = np.where(result<0, 1 ,result)

    return result2

In [73]:
vals = [[-3.612099949722249459e-01, 3.612099949722249459e-01],
[-3.522954985094583491e-01, 3.522954985094583491e-01],
[-3.436244428491525005e-01, 3.436244428491525005e-01],
[-3.352157198401093385e-01, 3.352157198401093385e-01],
[-3.270895620218176592e-01, 3.270895620218176592e-01],
[-3.192675460965237466e-01, 3.192675460965237466e-01],
[-3.117725646180832255e-01, 3.117725646180832255e-01],
[-3.046287572730904669e-01, 3.046287572730904669e-01]]

In [74]:
vals[0]

[-0.36120999497222495, 0.36120999497222495]

In [89]:
omega = 0.75
e_win = 0.1
occup = occup_0K(np.array(vals[0]))
pick_up_transition_pairs(np.array(vals[0]), omega, e_win, occup)

array([[0, 1]])

In [92]:
max(np.where(occup==1)[0])

0

In [93]:
inds_vb

array([0])

In [94]:
array = np.array([0.0,0.0])

In [95]:
array==0

array([ True,  True])

In [99]:
if array[0]==0:
    print('go')

go


In [101]:
np.where(array==0, 0.5, array)

array([0.5, 0.5])

In [104]:
8//3

2

In [105]:
8%3

2

In [108]:
nx, ny = (3, 2)
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(x, y, indexing='ij')
for i in range(nx):
    for j in range(ny):
        print(i)

0
0
1
1
2
2
